In [50]:
# Dependencies

import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Importing ALBERT
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from torch.optim import AdamW

# Processing

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [51]:
df = pd.read_csv(r"../Data/processedData.csv")

X_train, X_test, y_train, y_test = train_test_split(df[['processedMessage','fingers','tail']],
                                                    df['species_group'])

label_encoder = LabelEncoder()
label_encoder.fit(df['species_group'])

y_train = torch.tensor(to_categorical(label_encoder.transform(list(y_train)),num_classes=12))
y_test = torch.tensor(to_categorical(label_encoder.transform(list(y_test)),num_classes=12))

# ALBERT Transformer #

In [52]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=12)

X_train_encodings = tokenizer(list(X_train['processedMessage']), return_tensors='pt', padding=True, truncation=True, max_length=128)
X_test_encodings = tokenizer(list(X_test['processedMessage']), return_tensors='pt', padding=True, truncation=True, max_length=128)

train_dataset = TensorDataset(X_train_encodings['input_ids'], X_train_encodings['attention_mask'], y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = TensorDataset(X_test_encodings['input_ids'], X_test_encodings['attention_mask'], y_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

optimizer = AdamW(model.parameters())
loss_fn = torch.nn.BCEWithLogitsLoss()

C:\Users\suhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
model.train()

num_epochs = 10

for epoch in range(num_epochs):

    print(f"Epoch : {epoch+1}")

    for batch in train_loader:

        input_ids, attention_mask, labels = batch

        outputs = model(input_ids = input_ids, attention_mask = attention_mask, labels = labels)
        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch : 1
Epoch : 2
Epoch : 3
Epoch : 4
Epoch : 5
Epoch : 6
Epoch : 7
Epoch : 8
Epoch : 9
Epoch : 10


In [54]:
model.eval()

probability_array = []

with torch.no_grad():

    for batch in test_loader:

        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)

        probability_array.append(probabilities)

probability_array = np.array(torch.cat(probability_array, dim=0))
probability_array

array([[0.14419916, 0.04953301, 0.05554189, ..., 0.14445652, 0.06651963,
        0.04135079],
       [0.14419916, 0.04953301, 0.0555419 , ..., 0.14445652, 0.06651963,
        0.04135079],
       [0.14419919, 0.04953301, 0.0555419 , ..., 0.14445652, 0.06651963,
        0.04135079],
       ...,
       [0.14419916, 0.04953304, 0.0555419 , ..., 0.1444564 , 0.06651956,
        0.0413508 ],
       [0.14419916, 0.04953304, 0.0555419 , ..., 0.1444564 , 0.06651956,
        0.0413508 ],
       [0.14419916, 0.04953304, 0.0555419 , ..., 0.1444564 , 0.06651956,
        0.0413508 ]], dtype=float32)

In [55]:
from sklearn.metrics import classification_report

y_pred_classes = probability_array.argmax(axis = 1)
y_test_classes = ((y_test)).argmax(axis = 1)

print(classification_report(y_test_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00        14
           7       0.00      0.00      0.00        12
           8       0.00      0.00      0.00        14
           9       0.11      1.00      0.20        14
          10       0.00      0.00      0.00        16
          11       0.00      0.00      0.00        14

    accuracy                           0.11       125
   macro avg       0.01      0.08      0.02       125
weighted avg       0.01      0.11      0.02       125



C:\Users\suhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\suhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\suhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classificati